In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
from bs4 import BeautifulSoup

# Lista com as URLs dos imóveis
df = pd.read_csv('/content/' + csv_com_urls)
urls = df.urls.to_list()
count = 0
iteracao = 0
count_erros = 0
li = []

# Configurar o driver do Selenium
wd = webdriver.Chrome()

for i in range(0, len(df)):
    try:
        # Criação de um dicionário que armazenará os dados de cada imóvel
        elementos_imovel = {
            "url": [],
            "bairro": [],
            "aluguel": [],
            "condominio": [],
            "iptu": [],
            "seguro_incendio": [],
            "taxa_serviço": [],
            "total": [],
            "metragem": [],
            "quarto": [],
            "banheiro": [],
            "vaga_carro": [],
            "andar": [],
            "aceita_pet": [],
            "mobilia": [],
            "metro_prox": [],
            "latitude": [],
            "longitude": []
        }

        acessar_url = str(urls[i])  # Acessa a URL do imóvel
        wd.get(acessar_url)
        print('Iterando a URL ' + str(count) + ' de ' + str(len(urls)))
        print('Número de iterações: ' + str(iteracao))
        count += 1
        iteracao += 1

        elementos_imovel['url'].append(acessar_url)  # Guarda a URL acessada

        # Classe contendo os elementos de localização do imóvel
        WebDriverWait(wd, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "w0f64d-0.hBSKKA")))
        bairro_do_imovel = wd.find_elements_by_class_name("w0f64d-0.hBSKKA")
        print('o bairro é: ' + bairro_do_imovel[1].text)

        elementos_imovel['bairro'].append(bairro_do_imovel[1].text)

        # Classe contendo o quadro de preços do imóvel
        WebDriverWait(wd, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "sc-1mwqqtp-0.byvuTN")))
        preços = wd.find_elements_by_class_name("sc-1mwqqtp-0.byvuTN")

        x = 0
        while x < len(preços):
            print('os preços são: ' + preços[x].text)
            x += 1

        elementos_imovel["aluguel"].append(preços[0].text)
        elementos_imovel["condominio"].append(preços[1].text)
        elementos_imovel["iptu"].append(preços[2].text)
        elementos_imovel["seguro_incendio"].append(preços[3].text)
        elementos_imovel["taxa_serviço"].append(preços[4].text)
        elementos_imovel["total"].append(preços[5].text)

        # Classe contendo o quadro de features do imóvel
        WebDriverWait(wd, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "sc-bdVaJa.kxWnCU")))
        elementos_do_imovel = wd.find_elements_by_class_name("sc-bdVaJa.kxWnCU")

        x = 0
        while x < len(elementos_do_imovel):
            print('os elementos são: ' + elementos_do_imovel[x].text)
            x += 1

        elementos_imovel["metragem"].append(elementos_do_imovel[0].text)
        elementos_imovel["quarto"].append(elementos_do_imovel[1].text)
        elementos_imovel["banheiro"].append(elementos_do_imovel[2].text)
        elementos_imovel["vaga_carro"].append(elementos_do_imovel[3].text)
        elementos_imovel["andar"].append(elementos_do_imovel[4].text)
        elementos_imovel["aceita_pet"].append(elementos_do_imovel[5].text)
        elementos_imovel["mobilia"].append(elementos_do_imovel[6].text)
        elementos_imovel["metro_prox"].append(elementos_do_imovel[7].text)

        # Acessar o ícone do mapa e clicar para abrir o mapa com coordenadas
        try:
            map_icon = wd.find_element(By.CSS_SELECTOR, 'MuiBox-root mui-1r56sh1')
            action = ActionChains(wd)
            action.move_to_element(map_icon).click().perform()

            # Esperar que o mapa carregue
            time.sleep(5)
            nova_url = wd.current_url

            # Verificar se a URL contém o Google Maps e extrair coordenadas
            if "maps.googleapis.com" in nova_url:
                print("Mapa detectado na URL. Vamos buscar as coordenadas...")

                # Buscar a URL da imagem com as coordenadas
                response = wd.page_source
                soup = BeautifulSoup(response, 'html.parser')
                img_tag = soup.find('img', {'src': True})

                if img_tag:
                    img_src = img_tag['src']
                    if 'center=' in img_src and 'markers=' in img_src:
                        marker_coords = img_src.split('markers=')[1].split('&')[0]
                        latitude, longitude = marker_coords.split(',')
                        print(f"Latitude: {latitude}, Longitude: {longitude}")

                        # Armazenar as coordenadas
                        elementos_imovel["latitude"].append(latitude)
                        elementos_imovel["longitude"].append(longitude)

        except Exception as e:
            print(f"Erro ao tentar capturar coordenadas do mapa: {e}")

        # Estruturando o conteúdo em um arquivo CSV
        df = pd.DataFrame(elementos_imovel)
        li.append(df)
        frame = pd.concat(li, axis=0, ignore_index=True)
        frame.to_csv('dataset_features.csv', index=False)

    except Exception as e:
        print(f"Erro na iteração {iteracao}: {e}")

print('Processo concluído.')
